In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from tiingo import TiingoClient
import os
import time

# 设置Tiingo API密钥
os.environ['TIINGO_API_KEY'] = '5a4a9ea706e1c065068704ea40f1a78e17e78e46'

# 初始化Tiingo客户端
config = {}
config['session'] = True
config['api_key'] = os.environ['TIINGO_API_KEY']
client = TiingoClient(config)

# 更新后的XLK成分股列表（基于2025年8月搜索结果，约71只；从多个来源编译，包括PLTR、SMCI等新晋）
stock_tickers = [
    'AAPL', 'ACN', 'ADBE', 'ADI', 'AKAM', 'AMD', 'AMAT', 'ANET', 'APH', 'AVGO',
    'CDNS', 'CDW', 'CRWD', 'CRM', 'CSCO', 'CTSH', 'DDOG', 'DELL', 'ENPH', 'EPAM',
    'FICO', 'FFIV', 'FSLR', 'FTNT', 'GEN', 'GDDY', 'GLW', 'HPE', 'HPQ', 'IBM',
    'INTC', 'INTU', 'IT', 'JBL', 'KEYS', 'KLAC', 'LRCX', 'MCHP', 'MPWR', 'MSI',
    'MSFT', 'MU', 'NOW', 'NVDA', 'NXPI', 'ON', 'ORCL', 'PANW', 'PLTR', 'PTC',
    'QCOM', 'ROP', 'SMCI', 'SNPS', 'STX', 'SWKS', 'TEL', 'TER', 'TDY', 'TXN',
    'TYL', 'TRMB', 'VRSN', 'WDC', 'WDAY', 'ZBRA', 'XLK'
]  # 添加XLK ETF作为基准

# 数据期：从2019-12-01（为回看期）到当前
start_date = '2019-12-01'
end_date = datetime.today().strftime('%Y-%m-%d')

print(f"开始下载数据，日期范围：{start_date} 到 {end_date}")
print(f"股票数量：{len(stock_tickers)}")

# 用于存储所有数据的字典
all_data = {}
failed_tickers = []

# 分批获取数据，避免API限制
for i, ticker in enumerate(stock_tickers):
    try:
        print(f"正在下载 {ticker} ({i+1}/{len(stock_tickers)})")
        
        # 获取历史价格数据
        historical_prices = client.get_dataframe(
            ticker,
            startDate=start_date,
            endDate=end_date,
            frequency='daily'
        )
        
        if not historical_prices.empty:
            # 重置索引，确保日期是索引
            if 'date' in historical_prices.columns:
                historical_prices.set_index('date', inplace=True)
            
            # 存储调整后收盘价和成交量
            all_data[ticker] = {
                'adjClose': historical_prices['adjClose'] if 'adjClose' in historical_prices.columns else historical_prices['close'],
                'volume': historical_prices['volume'] if 'volume' in historical_prices.columns else np.nan
            }
        else:
            print(f"警告：{ticker} 没有返回数据")
            failed_tickers.append(ticker)
            
        # 添加延迟避免API限制
        time.sleep(0.1)
        
    except Exception as e:
        print(f"获取 {ticker} 数据时出错: {e}")
        failed_tickers.append(ticker)
        continue

print(f"\n数据下载完成！")
print(f"成功获取: {len(all_data)} 只股票")
print(f"失败: {len(failed_tickers)} 只股票")
if failed_tickers:
    print(f"失败的股票: {failed_tickers}")

# 创建调整后收盘价DataFrame
adj_close_data = {}
volume_data = {}

for ticker, data in all_data.items():
    adj_close_data[ticker] = data['adjClose']
    volume_data[ticker] = data['volume']

# 转换为DataFrame
adj_close = pd.DataFrame(adj_close_data)
volume = pd.DataFrame(volume_data)

# 确保索引是日期时间类型
adj_close.index = pd.to_datetime(adj_close.index)
volume.index = pd.to_datetime(volume.index)

# 排序索引
adj_close = adj_close.sort_index()
volume = volume.sort_index()

# 处理缺失值：向前/向后填充短缺（限5天），避免长缺口影响
adj_close = adj_close.ffill(limit=5).bfill(limit=5)
volume = volume.ffill(limit=5).bfill(limit=5)

# 保存为CSV
adj_close.to_csv('adj_close.csv')
volume.to_csv('volume.csv')

print(f"\n最终数据统计：")
print(f"日期范围：{adj_close.index[0].strftime('%Y-%m-%d')} 到 {adj_close.index[-1].strftime('%Y-%m-%d')}")
print(f"有效股票数：{len(adj_close.columns)}")
print(f"数据行数：{len(adj_close)}")
print(f"调整后收盘价数据已保存到 adj_close.csv")
print(f"成交量数据已保存到 volume.csv")

开始下载数据，日期范围：2019-12-01 到 2025-08-18
股票数量：67
正在下载 AAPL (1/67)
正在下载 ACN (2/67)
正在下载 ACN (2/67)
正在下载 ADBE (3/67)
正在下载 ADBE (3/67)
正在下载 ADI (4/67)
正在下载 ADI (4/67)
正在下载 AKAM (5/67)
正在下载 AKAM (5/67)
正在下载 AMD (6/67)
正在下载 AMD (6/67)
正在下载 AMAT (7/67)
正在下载 AMAT (7/67)
正在下载 ANET (8/67)
正在下载 ANET (8/67)
正在下载 APH (9/67)
正在下载 APH (9/67)
正在下载 AVGO (10/67)
正在下载 AVGO (10/67)
正在下载 CDNS (11/67)
正在下载 CDNS (11/67)
正在下载 CDW (12/67)
正在下载 CDW (12/67)
正在下载 CRWD (13/67)
正在下载 CRWD (13/67)
正在下载 CRM (14/67)
正在下载 CRM (14/67)
正在下载 CSCO (15/67)
正在下载 CSCO (15/67)
正在下载 CTSH (16/67)
正在下载 CTSH (16/67)
正在下载 DDOG (17/67)
正在下载 DDOG (17/67)
正在下载 DELL (18/67)
正在下载 DELL (18/67)
正在下载 ENPH (19/67)
正在下载 ENPH (19/67)
正在下载 EPAM (20/67)
正在下载 EPAM (20/67)
正在下载 FICO (21/67)
正在下载 FICO (21/67)
正在下载 FFIV (22/67)
正在下载 FFIV (22/67)
正在下载 FSLR (23/67)
正在下载 FSLR (23/67)
正在下载 FTNT (24/67)
正在下载 FTNT (24/67)
正在下载 GEN (25/67)
正在下载 GEN (25/67)
正在下载 GDDY (26/67)
正在下载 GDDY (26/67)
正在下载 GLW (27/67)
正在下载 GLW (27/67)
正在下载 HPE (28/67)
正在下载 HPE (28/67)
正

In [38]:
import requests
import json

# 替换为你的 Alpha Vantage API 密钥
api_key = 'BZETY055O7363C1W'

# API 端点：国债收益率（3 个月成熟期，对应 IRX）
url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=3month&apikey={api_key}'

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=4))  # 打印数据（JSON 格式）
else:
    print(f"Error: {response.status_code} - {response.text}")

{
    "name": "3-Month Treasury Constant Maturity Rate",
    "interval": "daily",
    "unit": "percent",
    "data": [
        {
            "date": "2025-08-14",
            "value": "4.3"
        },
        {
            "date": "2025-08-13",
            "value": "4.29"
        },
        {
            "date": "2025-08-12",
            "value": "4.33"
        },
        {
            "date": "2025-08-11",
            "value": "4.34"
        },
        {
            "date": "2025-08-08",
            "value": "4.32"
        },
        {
            "date": "2025-08-07",
            "value": "4.32"
        },
        {
            "date": "2025-08-06",
            "value": "4.32"
        },
        {
            "date": "2025-08-05",
            "value": "4.34"
        },
        {
            "date": "2025-08-04",
            "value": "4.35"
        },
        {
            "date": "2025-08-01",
            "value": "4.35"
        },
        {
            "date": "2025-07-31",
        

In [37]:
# 处理真实的IRX和VIX数据，替换合成数据
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import os

# 重新加载当前的数据
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

print("="*60)
print("使用真实数据替换合成数据")
print("="*60)

# 1. 获取并处理Alpha Vantage的IRX数据
print("1. 处理Alpha Vantage的IRX数据...")

api_key = 'BZETY055O7363C1W'
url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=3month&apikey={api_key}'

try:
    response = requests.get(url)
    if response.status_code == 200:
        irx_data = response.json()
        
        if 'data' in irx_data:
            # 转换为DataFrame
            irx_df = pd.DataFrame(irx_data['data'])
            irx_df['date'] = pd.to_datetime(irx_df['date'])
            irx_df.set_index('date', inplace=True)
            irx_df['value'] = pd.to_numeric(irx_df['value'], errors='coerce')
            
            # 筛选我们需要的日期范围
            start_datetime = pd.to_datetime('2019-12-01')
            end_datetime = pd.to_datetime('2025-08-16')
            irx_df = irx_df[(irx_df.index >= start_datetime) & (irx_df.index <= end_datetime)]
            irx_df = irx_df.sort_index()
            
            print(f"IRX数据点数: {len(irx_df)}")
            
            # 处理时区问题 - 统一为无时区
            if adj_close.index.tz is not None:
                adj_close.index = adj_close.index.tz_localize(None)
            if irx_df.index.tz is not None:
                irx_df.index = irx_df.index.tz_localize(None)
            
            # 将真实IRX数据替换到adj_close中
            for date in adj_close.index:
                # 找到最近的IRX数据日期
                available_dates = irx_df.index[irx_df.index <= date]
                if len(available_dates) > 0:
                    closest_date = available_dates.max()
                    adj_close.loc[date, '^IRX'] = irx_df.loc[closest_date, 'value']
            
            irx_updated_count = adj_close['^IRX'].notna().sum()
            print(f"✅ IRX真实数据替换成功，共{irx_updated_count}个交易日")
            print(f"   数据范围: {irx_df['value'].min():.3f}% - {irx_df['value'].max():.3f}%")
            print(f"   最新值: {adj_close['^IRX'].iloc[-1]:.3f}%")
        else:
            print("❌ Alpha Vantage IRX数据格式不正确")
    else:
        print(f"❌ Alpha Vantage请求失败: {response.status_code}")
        
except Exception as e:
    print(f"❌ IRX数据处理失败: {e}")

# 2. 处理VIX数据
print(f"\n2. 处理VIX数据...")

vix_file = 'VIX.csv'
vix_success = False

if os.path.exists(vix_file):
    try:
        print(f"找到VIX文件: {vix_file}")
        vix_df = pd.read_csv(vix_file)
        
        # 适配中文列名
        column_mapping = {
            '日期': 'date',
            '收盘': 'close',
            '开盘': 'open',
            '高': 'high',
            '低': 'low'
        }
        
        # 重命名列
        vix_df = vix_df.rename(columns=column_mapping)
        
        if 'date' in vix_df.columns and 'close' in vix_df.columns:
            # 处理日期
            vix_df['date'] = pd.to_datetime(vix_df['date'], errors='coerce')
            vix_df = vix_df.dropna(subset=['date'])
            vix_df.set_index('date', inplace=True)
            
            # 清理价格数据
            if vix_df['close'].dtype == 'object':
                vix_df['close'] = vix_df['close'].astype(str).str.replace(',', '').str.replace('%', '')
            vix_df['close'] = pd.to_numeric(vix_df['close'], errors='coerce')
            
            # 筛选日期范围
            start_datetime = pd.to_datetime('2019-12-01')
            end_datetime = pd.to_datetime('2025-08-18')
            vix_df = vix_df[(vix_df.index >= start_datetime) & (vix_df.index <= end_datetime)]
            vix_df = vix_df.sort_index()
            
            print(f"VIX数据点数: {len(vix_df)}")
            
            # 处理时区问题 - 统一为无时区
            if vix_df.index.tz is not None:
                vix_df.index = vix_df.index.tz_localize(None)
            
            # 将真实VIX数据替换到adj_close中
            for date in adj_close.index:
                # 找到最近的VIX数据日期
                available_dates = vix_df.index[vix_df.index <= date]
                if len(available_dates) > 0:
                    closest_date = available_dates.max()
                    adj_close.loc[date, '^VIX'] = vix_df.loc[closest_date, 'close']
            
            vix_updated_count = adj_close['^VIX'].notna().sum()
            print(f"✅ VIX真实数据替换成功，共{vix_updated_count}个交易日")
            print(f"   数据范围: {vix_df['close'].min():.2f} - {vix_df['close'].max():.2f}")
            print(f"   最新值: {adj_close['^VIX'].iloc[-1]:.2f}")
            vix_success = True
        else:
            print(f"❌ 无法识别VIX文件列结构")
            
    except Exception as e:
        print(f"❌ VIX数据处理失败: {e}")
else:
    print("❌ 未找到VIX.csv文件")

# 3. 从volume数据中删除VIX和IRX（如果存在）
print(f"\n3. 清理volume数据...")

volume_updated = False
if '^VIX' in volume.columns:
    volume = volume.drop(columns=['^VIX'])
    print("✅ 已从volume数据中删除VIX")
    volume_updated = True

if '^IRX' in volume.columns:
    volume = volume.drop(columns=['^IRX'])
    print("✅ 已从volume数据中删除IRX")
    volume_updated = True

if not volume_updated:
    print("ℹ️ VIX和IRX已经不在volume数据中")

# 4. 保存更新后的数据
print(f"\n4. 保存更新后的数据...")

adj_close.to_csv('adj_close.csv')
volume.to_csv('volume.csv')

print("✅ adj_close.csv 已更新（使用真实的IRX和VIX数据）")
print("✅ volume.csv 已更新（删除了VIX和IRX的volume数据）")

# 5. 数据质量检查
print(f"\n5. 更新后的数据统计:")
print(f"   adj_close形状: {adj_close.shape}")
print(f"   volume形状: {volume.shape}")

if '^IRX' in adj_close.columns:
    irx_real_count = adj_close['^IRX'].notna().sum()
    irx_latest = adj_close['^IRX'].iloc[-1]
    irx_mean = adj_close['^IRX'].mean()
    irx_std = adj_close['^IRX'].std()
    print(f"   IRX真实数据: {irx_real_count}点")
    print(f"      最新值: {irx_latest:.3f}%")
    print(f"      平均值: {irx_mean:.3f}%")
    print(f"      标准差: {irx_std:.3f}%")

if '^VIX' in adj_close.columns:
    vix_real_count = adj_close['^VIX'].notna().sum()
    vix_latest = adj_close['^VIX'].iloc[-1]
    vix_mean = adj_close['^VIX'].mean()
    vix_std = adj_close['^VIX'].std()
    print(f"   VIX真实数据: {vix_real_count}点")
    print(f"      最新值: {vix_latest:.2f}")
    print(f"      平均值: {vix_mean:.2f}")
    print(f"      标准差: {vix_std:.2f}")

# 验证数据差异
print(f"\n6. 验证数据变化:")
print("   前5行VIX值:", adj_close['^VIX'].head().tolist())
print("   前5行IRX值:", adj_close['^IRX'].head().tolist())

print(f"\n✨ 真实数据替换完成！")
print("   - IRX数据来自Alpha Vantage API")
print("   - VIX数据来自您下载的investing.com文件")
print("   - 已从volume数据中移除VIX和IRX列")
print("   - 所有合成数据已被真实历史数据替换")

使用真实数据替换合成数据
1. 处理Alpha Vantage的IRX数据...
IRX数据点数: 1489
✅ IRX真实数据替换成功，共1425个交易日
   数据范围: 0.000% - 5.630%
   最新值: 4.300%

2. 处理VIX数据...
找到VIX文件: VIX.csv
VIX数据点数: 1470
IRX数据点数: 1489
✅ IRX真实数据替换成功，共1425个交易日
   数据范围: 0.000% - 5.630%
   最新值: 4.300%

2. 处理VIX数据...
找到VIX文件: VIX.csv
VIX数据点数: 1470
✅ VIX真实数据替换成功，共1434个交易日
   数据范围: 11.86 - 82.69
   最新值: 15.09

3. 清理volume数据...
ℹ️ VIX和IRX已经不在volume数据中

4. 保存更新后的数据...
✅ adj_close.csv 已更新（使用真实的IRX和VIX数据）
✅ volume.csv 已更新（删除了VIX和IRX的volume数据）

5. 更新后的数据统计:
   adj_close形状: (1434, 69)
   volume形状: (1434, 67)
   IRX真实数据: 1425点
      最新值: 4.300%
      平均值: 2.765%
      标准差: 2.308%
   VIX真实数据: 1434点
      最新值: 15.09
      平均值: 21.16
      标准差: 8.06

6. 验证数据变化:
   前5行VIX值: [14.91, 15.96, 14.8, 14.52, 13.62]
   前5行IRX值: [1.6, 1.57, 1.55, 1.54, 1.53]

✨ 真实数据替换完成！
   - IRX数据来自Alpha Vantage API
   - VIX数据来自您下载的investing.com文件
   - 已从volume数据中移除VIX和IRX列
   - 所有合成数据已被真实历史数据替换
✅ VIX真实数据替换成功，共1434个交易日
   数据范围: 11.86 - 82.69
   最新值: 15.09

3. 清理volume数据...
ℹ️ VIX和IR

In [40]:
print(f"数据下载完成。日期范围：{adj_close.index[0]} 到 {adj_close.index[-1]}。股票数：{len(adj_close.columns)}。行数：{len(adj_close)}")

数据下载完成。日期范围：2019-12-02 00:00:00 到 2025-08-15 00:00:00。股票数：69。行数：1434


In [ ]:
import pandas as pd
import numpy as np

def calculate_momentum_scores(adj_close, volume, daily_returns, previous_date, lookback_months, method='simple'):
    """
    计算给定日期所有有效股票的动量分数。
    
    参数:
    - adj_close: 调整后收盘价DataFrame (从步骤1的CSV加载)
    - volume: 成交量DataFrame
    - daily_returns: 日回报DataFrame (adj_close.pct_change())
    - previous_date: 计算日期（上个月末，pd.Timestamp）
    - lookback_months: 回看期（月，例如3、6、12）
    - method: 'simple'（简单）、'risk_adjusted'（风险调整）或'volume_weighted'（成交量加权）
    
    返回:
    - 以ticker为索引的分数Series（越高越好动量）
    
    示例使用:
    scores = calculate_momentum_scores(adj_close, volume, daily_returns, pd.Timestamp('2025-07-31'), 6, 'simple')
    """
    
    # 处理时区问题 - 统一移除时区信息
    if adj_close.index.tz is not None:
        adj_close_tz_naive = adj_close.copy()
        adj_close_tz_naive.index = adj_close.index.tz_localize(None)
    else:
        adj_close_tz_naive = adj_close
    
    if volume.index.tz is not None:
        volume_tz_naive = volume.copy()
        volume_tz_naive.index = volume.index.tz_localize(None)
    else:
        volume_tz_naive = volume
    
    if daily_returns.index.tz is not None:
        daily_returns_tz_naive = daily_returns.copy()
        daily_returns_tz_naive.index = daily_returns.index.tz_localize(None)
    else:
        daily_returns_tz_naive = daily_returns
    
    # 确保previous_date也是无时区的
    if hasattr(previous_date, 'tz') and previous_date.tz is not None:
        previous_date = previous_date.tz_localize(None)
    
    # 找到回看起始日期（近似N个月前，取最近交易日）
    lookback_start = previous_date - pd.DateOffset(months=lookback_months)
    available_dates = adj_close_tz_naive.index[adj_close_tz_naive.index >= lookback_start]
    if len(available_dates) == 0:
        print(f"警告: 没有找到 {lookback_start} 之后的数据")
        return pd.Series()
    lookback_start = available_dates[0]
    
    # 提取看回期数据，只包括无NaN的股票（排除XLK、^VIX、^IRX）
    close_lookback = adj_close_tz_naive.loc[lookback_start:previous_date]
    
    # 排除指数和ETF，只保留个股
    exclude_symbols = ['XLK', '^VIX', '^IRX']
    valid_stocks = []
    for col in close_lookback.columns:
        if col not in exclude_symbols and close_lookback[col].notna().all():
            valid_stocks.append(col)
    
    valid_stocks = pd.Index(valid_stocks)
    
    if len(valid_stocks) < 10:  # 最小股票数阈值，避免无效计算
        print(f"警告: 有效股票数量过少 ({len(valid_stocks)})，无法进行有效计算")
        return pd.Series()
    
    # 基础动量：价格变化率
    try:
        close_t = adj_close_tz_naive.loc[previous_date, valid_stocks]
        close_tk = adj_close_tz_naive.loc[lookback_start, valid_stocks]
        mom = close_t / close_tk - 1
    except KeyError as e:
        print(f"错误: 日期 {previous_date} 或 {lookback_start} 不在数据中")
        return pd.Series()
    
    if method == 'simple':
        scores = mom
    elif method == 'risk_adjusted':
        # 计算年化波动率
        daily_ret_lb = daily_returns_tz_naive.loc[lookback_start:previous_date, valid_stocks]
        vol = daily_ret_lb.std() * np.sqrt(252)  # 年化
        scores = mom / vol.replace(0, np.nan)  # 避免除零，用NaN替换
    elif method == 'volume_weighted':
        # 基准成交量：过去12个月或可用期
        baseline_start = previous_date - pd.DateOffset(months=12)
        available_baseline_dates = adj_close_tz_naive.index[adj_close_tz_naive.index >= baseline_start]
        if len(available_baseline_dates) == 0:
            baseline_start = adj_close_tz_naive.index[0]  # 使用最早可用日期
        else:
            baseline_start = available_baseline_dates[0]
            
        # 只选择在volume数据中存在的股票
        volume_valid_stocks = valid_stocks.intersection(volume_tz_naive.columns)
        
        vol_lb = volume_tz_naive.loc[lookback_start:previous_date, volume_valid_stocks].mean()
        vol_baseline = volume_tz_naive.loc[baseline_start:previous_date, volume_valid_stocks].mean()
        volume_factor = vol_lb / vol_baseline  # 相对成交量
        
        # 只对有成交量数据的股票计算分数
        scores = pd.Series(index=valid_stocks, dtype=float)
        for stock in volume_valid_stocks:
            if stock in mom.index and not pd.isna(volume_factor[stock]) and volume_factor[stock] > 0:
                scores[stock] = mom[stock] * volume_factor[stock]
            elif stock in mom.index:
                scores[stock] = mom[stock]  # 如果没有成交量数据，使用基础动量
        
        scores = scores.dropna()
    else:
        raise ValueError(f"无效方法: {method}")
    
    return scores.dropna()  # 丢弃任何剩余NaN

In [44]:
# 加载数据（从步骤1）
adj_close = pd.read_csv('adj_close.csv', index_col=0, parse_dates=True)
volume = pd.read_csv('volume.csv', index_col=0, parse_dates=True)

# 处理时区问题 - 统一移除时区信息
if adj_close.index.tz is not None:
    adj_close.index = adj_close.index.tz_localize(None)
if volume.index.tz is not None:
    volume.index = volume.index.tz_localize(None)

# 修复FutureWarning - 明确指定fill_method=None
daily_returns = adj_close.pct_change(fill_method=None)

print("数据加载完成:")
print(f"adj_close形状: {adj_close.shape}")
print(f"volume形状: {volume.shape}")
print(f"日期范围: {adj_close.index[0]} 到 {adj_close.index[-1]}")

# 测试示例：2025年7月末，6个月看回，简单方法
test_date = pd.Timestamp('2025-07-31')  # 假设这是月末；如果无数据，取最近

# 确保test_date是无时区的
if hasattr(test_date, 'tz') and test_date.tz is not None:
    test_date = test_date.tz_localize(None)

if test_date not in adj_close.index:
    available_dates = adj_close.index[adj_close.index <= test_date]
    if len(available_dates) > 0:
        test_date = available_dates[-1]  # 取最近的一个交易日
    else:
        test_date = adj_close.index[-1]  # 如果没有合适的日期，取最新日期

print(f"\n使用测试日期: {test_date}")

# 简单动量分数
print("\n=== 简单动量分数 ===")
scores_simple = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'simple')
if len(scores_simple) > 0:
    print(f"简单动量分数（前5）：\n{scores_simple.sort_values(ascending=False).head(5)}")
else:
    print("简单动量计算失败")

# 风险调整动量分数
print("\n=== 风险调整动量分数 ===")
scores_risk = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'risk_adjusted')
if len(scores_risk) > 0:
    print(f"风险调整动量分数（前5）：\n{scores_risk.sort_values(ascending=False).head(5)}")
else:
    print("风险调整动量计算失败")

# 成交量加权动量分数
print("\n=== 成交量加权动量分数 ===")
scores_vol = calculate_momentum_scores(adj_close, volume, daily_returns, test_date, 6, 'volume_weighted')
if len(scores_vol) > 0:
    print(f"成交量加权动量分数（前5）：\n{scores_vol.sort_values(ascending=False).head(5)}")
else:
    print("成交量加权动量计算失败")

print(f"\n=== 测试完成 ===")
print(f"有效股票数量: {len(scores_simple)} (简单), {len(scores_risk)} (风险调整), {len(scores_vol)} (成交量加权)")

数据加载完成:
adj_close形状: (1434, 69)
volume形状: (1434, 67)
日期范围: 2019-12-02 00:00:00 到 2025-08-15 00:00:00

使用测试日期: 2025-07-31 00:00:00

=== 简单动量分数 ===
简单动量分数（前5）：
SMCI    1.067672
PLTR    0.919627
STX     0.651139
WDC     0.601384
AMD     0.520569
dtype: float64

=== 风险调整动量分数 ===
风险调整动量分数（前5）：
APH     1.414457
STX     1.321326
PLTR    1.142661
TEL     1.130787
WDC     1.103285
dtype: float64

=== 成交量加权动量分数 ===
成交量加权动量分数（前5）：
SMCI    1.201536
PLTR    1.062787
STX     0.803889
WDC     0.691212
APH     0.569634
dtype: float64

=== 测试完成 ===
有效股票数量: 66 (简单), 66 (风险调整), 66 (成交量加权)


In [46]:
# 动量分析结果总结
print("="*70)
print("动量分析结果总结")
print("="*70)

print("✅ 时区问题修复完成:")
print("   - 统一移除了所有DataFrame的时区信息")
print("   - 修复了 'Cannot compare tz-naive and tz-aware datetime-like objects' 错误")
print("   - 修复了 FutureWarning 关于 pct_change() 方法的警告")

print(f"\n📊 动量计算结果 (测试日期: {test_date.strftime('%Y-%m-%d')}):")

# 创建综合比较表
momentum_comparison = pd.DataFrame({
    '简单动量': scores_simple,
    '风险调整动量': scores_risk,
    '成交量加权动量': scores_vol
}).fillna(0)

print(f"\n🏆 各方法Top 5股票:")
print("\n简单动量 (前5):")
for i, (ticker, score) in enumerate(scores_simple.sort_values(ascending=False).head(10).items(), 1):
    print(f"  {i}. {ticker}: {score:.4f} ({score*100:.2f}%)")

print("\n风险调整动量 (前5):")
for i, (ticker, score) in enumerate(scores_risk.sort_values(ascending=False).head(5).items(), 1):
    print(f"  {i}. {ticker}: {score:.4f}")

print("\n成交量加权动量 (前5):")
for i, (ticker, score) in enumerate(scores_vol.sort_values(ascending=False).head(5).items(), 1):
    print(f"  {i}. {ticker}: {score:.4f}")

# 分析一致性
top_simple = set(scores_simple.sort_values(ascending=False).head(10).index)
top_risk = set(scores_risk.sort_values(ascending=False).head(10).index)
top_vol = set(scores_vol.sort_values(ascending=False).head(10).index)

common_all = top_simple & top_risk & top_vol
common_simple_risk = top_simple & top_risk
common_simple_vol = top_simple & top_vol
common_risk_vol = top_risk & top_vol

print(f"\n🔍 Top 10 一致性分析:")
print(f"   三种方法都入选: {len(common_all)} 只股票 {list(common_all)}")
print(f"   简单+风险调整: {len(common_simple_risk)} 只股票")
print(f"   简单+成交量加权: {len(common_simple_vol)} 只股票")
print(f"   风险调整+成交量加权: {len(common_risk_vol)} 只股票")

print(f"\n💡 重要观察:")
print("   - SMCI在简单动量和成交量加权中都排第一")
print("   - PLTR在所有三种方法中都表现优秀")
print("   - STX和WDC在多种方法中都有强势表现")
print("   - 风险调整方法识别出APH作为高质量动量股票")

print(f"\n✨ 动量计算系统正常运行！")
print(f"   - 成功计算了66只股票的动量分数")
print(f"   - 三种不同方法提供了多角度的动量分析")
print(f"   - 系统可用于投资组合构建和股票筛选")

动量分析结果总结
✅ 时区问题修复完成:
   - 统一移除了所有DataFrame的时区信息
   - 修复了 'Cannot compare tz-naive and tz-aware datetime-like objects' 错误
   - 修复了 FutureWarning 关于 pct_change() 方法的警告

📊 动量计算结果 (测试日期: 2025-07-31):

🏆 各方法Top 5股票:

简单动量 (前5):
  1. SMCI: 1.0677 (106.77%)
  2. PLTR: 0.9196 (91.96%)
  3. STX: 0.6511 (65.11%)
  4. WDC: 0.6014 (60.14%)
  5. AMD: 0.5206 (52.06%)
  6. APH: 0.5114 (51.14%)
  7. ORCL: 0.5010 (50.10%)
  8. NVDA: 0.4816 (48.16%)
  9. TEL: 0.4025 (40.25%)
  10. JBL: 0.3754 (37.54%)

风险调整动量 (前5):
  1. APH: 1.4145
  2. STX: 1.3213
  3. PLTR: 1.1427
  4. TEL: 1.1308
  5. WDC: 1.1033

成交量加权动量 (前5):
  1. SMCI: 1.2015
  2. PLTR: 1.0628
  3. STX: 0.8039
  4. WDC: 0.6912
  5. APH: 0.5696

🔍 Top 10 一致性分析:
   三种方法都入选: 8 只股票 ['TEL', 'NVDA', 'WDC', 'SMCI', 'ORCL', 'STX', 'PLTR', 'APH']
   简单+风险调整: 8 只股票
   简单+成交量加权: 10 只股票
   风险调整+成交量加权: 8 只股票

💡 重要观察:
   - SMCI在简单动量和成交量加权中都排第一
   - PLTR在所有三种方法中都表现优秀
   - STX和WDC在多种方法中都有强势表现
   - 风险调整方法识别出APH作为高质量动量股票

✨ 动量计算系统正常运行！
   - 成功计算了66只股票的动量分数
   - 三种不